## Hardware

I hit my the limit of my GPUs a lot during training this model. I was consistently at 99%+ memory utilization on both GPUs, even after the many memory optimizations below. I was training on two 2070 Supers with nvlink during my training run. I had a 2080TI, but found that this was not sufficient. My memory optimizations are listed below. 
Checkpointing
Checkpointing⁵ is the act of not storing the input to a layer when doing the forward pass, and instead calculating it during the backward pass. This is done to save memory on the GPU. Wait, what does that even mean?!? Well, lets look at the chain rule, the derivative of f(g(x)) is f'(g(x))g'(x), notice that the "input" or g(x) is needed to calculate f'(g(x)), checkpointing is simply not storing g(x) for the backward pass when we calculate f(g(x)) during the forward pass.
Using checkpointing was a sad, and very upsetting thing for me. It turned out that I had to use more checkpointing than I originally expected, and checkpointing the smaller fp16 layers did not give the same memory savings as I had hoped. This meant I had to checkpoint much of my network, in order to get the the "magical" batch size of 3 above In the end this meant that I ended up nullifying much of speed advantage I got from training in fp16. Still it was a very great learning experience. If I ever get a larger GPU I would expect the training time to decrease due to the removal of much of the checkpoints.
Model Parallelism

I used a fairly odd solution to my memory problems in the form of model parallelism. This is because I determined that the biggest problem for me was the size of my model, and not the input data or speed. I decided to split my model based on the generator discriminator pairings, because that seemed to require the least information transfer between GPUs. Disclaimer, I did have access to nvlink, which many people may not have access to that was able to speed up the transfer of data between GPUs. This did require me to make copies of my 6 fp16 images on both GPUS, but was much more effective for me because the majority of my GPU memory was going to holding Adam moving averages and non-checkpointed layer's backward passes in memory.
Splitting of Linear Layers

One of the issues I was running into was running out of memory during the optimizer step. This was because of one linear layer of U-GAT-IT having approximately 256*2,000,000 parameters. Holding all of the parameters in memory to do the Adam step calculation meant that I would run out of memory every time I got to this point. So, in order to decrease memory consumption during this step, I split the linear layer into into 32 different linear layers, pass inputs into each, then concatenated the results, before passing that to the next layer as normal.
Zeroing Adam

Normally in PyTorch you zero the gradients after an optimizer step. The above splitting of linear layers was more effective, but I also made a small change to Adam, to go through the parameters smallest to largest, clearing the gradients along the way so that more memory was available for the largest layer's Adam step.
Small Memory optimizations

It is important when working in a memory limited environment that variables are garbage collected early. Especially when that variable is large! Early garbage collection of variables by wrapping memory intensive operations in functions, so they went out of scope was my preferred way of handling this situation. That allowed the code to in general be more expressive and concise.
<Example of Memory Optimization through function>